In [1]:
# @title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License

# Apache Beam RunInference with Vertex AI

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/apache/beam/blob/master/examples/notebooks/beam-ml/run_inference_vertex_ai.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/apache/beam/blob/master/examples/notebooks/beam-ml/run_inference_vertex_ai.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png" />View source on GitHub</a>
  </td>
</table>


## Before you begin
Set up your environment and download dependencies.

### Install Apache Beam
To use RunInference with the built-in Vertex AI model handler, install Apache Beam version 2.50.0 or later.

**Note:** at time of writing 2.50.0 is in the release process and is not available, so this notebook uses a release candidate build.

In [2]:
!pip install protobuf --quiet
!pip install apache_beam[gcp,interactive]==2.50.0rc1 --quiet
# Enforce shapely < 2.0.0 to avoid an issue with google.aiplatform
!pip install shapely==1.7.1 --quiet

### Authenticate with Google Cloud
This notebook relies on having a Vertex AI endpoint deployed to Google Cloud. To use your Google Cloud account, authenticate this notebook.

In [3]:
from google.colab import auth
auth.authenticate_user()

### Import dependencies and set up your bucket
Use the following code to import dependencies and to set up your Google Cloud Storage bucket.

Replace `PROJECT_ID`, `LOCATION_NAME`, and `ENDPOINT_ID` with the ID of your project, the GCP region where your model is deployed, and the ID of your Vertex AI endpoint.

**Important**: If an error occurs, restart your runtime.

In [4]:
import requests
from typing import Iterable
from typing import List
from typing import Tuple
from google.cloud import aiplatform

import apache_beam as beam
import tensorflow as tf
from apache_beam.ml.inference.base import KeyedModelHandler
from apache_beam.ml.inference.base import PredictionResult
from apache_beam.ml.inference.base import RunInference
from apache_beam.ml.inference.vertex_ai_inference import VertexAIModelHandlerJSON

project = "PROJECT_ID"
location = "LOCATION_NAME"
endpoint_id = "ENDPOINT_ID"


2023-08-23 19:03:47.352583: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 19:03:48.877610: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Query your Endpoint

This step checks that your model is deployed to your Vertex AI endpoint

In [5]:
aiplatform.init(project=project, location=location)
endpoint = aiplatform.Endpoint(endpoint_name=endpoint_id)
# You can get more metadata if desired by removing [0].display_name
endpoint.list_models()[0].display_name

'hello_custom'

### Get and pre-process an example image

This step shows how to preprocess an input to match what your model expects. We'll use an image of sunflowers to test the model and trim it to a 128x128 size to match the model's expectations and output the image as a list.

In [6]:
IMG_WIDTH = 128
IMG_URL = "https://www.chicagobotanic.org/sites/default/files/images/sunflowers/sunflower_big1.jpg"

def download_image(image_url: str) -> bytes:
  return requests.get(image_url).content

def preprocess_image(data: bytes) -> List[float]:
  """Preprocess the image, resizing it and normalizing it before
  converting to a list.
  """
  image = tf.io.decode_jpeg(data, channels=3)
  image = tf.image.resize_with_pad(image, IMG_WIDTH, IMG_WIDTH)
  image = image / 255
  return image.numpy().tolist()

img = download_image(IMG_URL)
example = preprocess_image(img)

2023-08-23 19:03:56.691395: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-23 19:03:56.728315: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-23 19:03:56.728545: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## Run the pipeline
Use the following code to run the pipeline and get an image classification and a probability from the Vertex AI endpoint.

In [7]:
# Column labels for the output probabilities.
COLUMNS = ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

class PostProcessor(beam.DoFn):
  def process(self, element: PredictionResult) -> Iterable[str]:
    prediction_vals = element.inference
    index = prediction_vals.index(max(prediction_vals))
    yield str(COLUMNS[index]) + " (" + str(max(prediction_vals)) + ")"

model_handler = VertexAIModelHandlerJSON(endpoint_id=endpoint_id, project=project, location=location)

with beam.Pipeline() as p:
    _ = (p | beam.Create([example])
           | RunInference(model_handler)
           | beam.ParDo(PostProcessor())
           | beam.Map(print)
        )

sunflowers (0.999636769)


## Use a keyed model handler
To use a keyed model handler, use `KeyedModelHandler` with Vertex AI by using `VertexAIModelHandlerJSON`.

By default, the `ModelHandler` does not expect a key.

* If you know that keys are associated with your examples, use `beam.KeyedModelHandler` to wrap the model handler.
* If you don't know whether keys are associated with your examples, use `beam.MaybeKeyedModelHandler`.

In [8]:
class PostProcessorKeyed(beam.DoFn):
  def process(self, element: Tuple[str, PredictionResult]) -> Iterable[str]:
    img_name, prediction_result = element
    prediction_vals = prediction_result.inference
    index = prediction_vals.index(max(prediction_vals))
    yield img_name + ": " + str(COLUMNS[index]) + " (" + str(
        max(prediction_vals)) + ")"

keyed_model_handler = KeyedModelHandler(VertexAIModelHandlerJSON(endpoint_id=endpoint_id, project=project, location=location))
with beam.Pipeline() as p:
    _ = (p | 'CreateExamples' >> beam.Create([example])
           | beam.Map(lambda image: (IMG_URL, image))
           | RunInference(keyed_model_handler)
           | beam.ParDo(PostProcessorKeyed())
           | beam.Map(print)
        )

https://www.chicagobotanic.org/sites/default/files/images/sunflowers/sunflower_big1.jpg: sunflowers (0.999636769)
